In [3]:
import json, os
LABELS_PATH = "Models/labels.json"
with open(LABELS_PATH, "r") as f:
    CLASS_NAMES = json.load(f)


In [4]:
import tkinter as tk
from PIL import ImageTk,Image, ImageDraw #importing pillow
import numpy as np #importing numpy
import cv2 #importing opencv
import joblib
import tensorflow as tf
import importlib, ensemble_utils_v2
importlib.reload(ensemble_utils_v2)
from ensemble_utils_v2 import segment_and_extract_digits, ensemble_predict



## Create The Main Window

In [5]:
# EXACT class order used at training time (example; EDIT to match your notebook!)
CLASS_NAMES = ['0','1','2','3','4','5','6','7','8','9']


In [6]:
win=tk.Tk()

# Main window

w, h = 1000, 400
fontButton='fixedsys 20 normal'
fontLabel='fixedsys 24 bold'
count=0

cnn = tf.keras.models.load_model("Models/mnist_CNN.keras")
dense = tf.keras.models.load_model("Models/mnist_DNN.keras")

img = Image.new('RGB', (w, h), (0, 0, 0))
ImgDraw = ImageDraw.Draw(img)

def eventFunction(event):
    
    x=event.x
    y=event.y
   
    x1=x-15
    y1=y-15
    x2=x+15
    y2=y+15
    
    canvas.create_oval((x1, y1, x2, y2), fill='black')
    ImgDraw.ellipse((x1,y1,x2,y2), fill='white')

def save():
    
    print('Save Pressed!')
    
    global count
    
    imgArray=np.array(img)
    imgArray=cv2.resize(imgArray, (28,28))
    
    cv2.imwrite('data2/'+ str(count) + '.jpg',imgArray )
    count+=1
    
def clear():
    
    global img, ImgDraw
    canvas.delete('all')
    img = Image.new('RGB', (w, h), (0, 0, 0))
    ImgDraw = ImageDraw.Draw(img)
    labelStatus.config(text=f"Predicted: None  |  Mode: {selected_model.get()}")

def label_map(index):
    """
    Convert class index to actual character.
    0–9  => '0'–'9'
    10–35 => 'A'–'Z'
    """
    if 0 <= index <= 9:
        return str(index)
    elif 10 <= index < 36:
        return chr(ord('A') + (index - 10))
    else:
        return '?'

Predict With Multiple Model

In [7]:
def predict():
    imgArray = np.array(img)
    digits, _, _ = segment_and_extract_digits(imgArray)


    if not digits:
        labelStatus.config(text=f"No digits detected!  |  Mode: {selected_model.get()}")
        print("DEBUG: 0 digits found")
        return

    model_type = selected_model.get()
    print(f"DEBUG: Using model: {model_type}, {len(digits)} digit(s) found")

    final_number = ""

    for d in digits:
        pred = "?"  # default fallback

        try:
            # ---------------- CNN ----------------
            if model_type == "CNN":
                probs = cnn.predict(d.reshape(1, 28, 28, 1), verbose=0)[0]
                pred = int(np.argmax(probs))

            # ---------------- DNN ----------------
            elif model_type == "DNN" and dense is not None:
                flat = d.reshape(1, -1)
                probs = dense.predict(flat, verbose=0)[0]
                pred = int(np.argmax(probs))

            # ---------------- Ensemble ----------------
            elif model_type == "Ensemble":
                from ensemble_utils_v2 import ensemble_predict
                pred, combined_probs = ensemble_predict(d, cnn, dense)

                # Handle invalid or failed ensemble prediction
                if pred == "?" or pred is None:
                    print("⚠️ Ensemble returned invalid prediction for this digit.")
                    pred = "?"
                else:
                    print(
                        f"DEBUG: Ensemble → CNN={np.argmax(cnn.predict(d.reshape(1,28,28,1), verbose=0)[0])}, "
                        f"Dense={np.argmax(dense.predict(d.reshape(1,-1), verbose=0)[0])}, "
                        f"KNN={int(knn.predict(d.reshape(1,-1))[0])} → Final={pred}"
                    )

        except Exception as e:
            print(f"⚠️ Prediction Error in {model_type}: {e}")
            pred = "?"

        # Append this digit’s result
        final_number += str(pred)

    # Update GUI label with final result
    labelStatus.config(text=f"Predicted: {final_number}  |  Mode: {model_type}")
    print(f"Predicted: {final_number}")


Predict CNN only

In [8]:
# def predict():
#     imgArray = np.array(img)
#     digits, _, _ = segment_and_extract_digits(imgArray)
#     if not digits:
#         labelStatus.config(text="No digits detected!"); print("DEBUG: 0"); return

#     out = []
#     for d in digits:
#         p = predict_cnn_only(d, cnn)   # softmax shape = (N_classes,)
#         p = p[:10] if len(p) > 10 else p  # restrict to digits for this test
#         idx = int(np.argmax(p))
#         out.append(CLASS_NAMES[idx])

#     labelStatus.config(text=f"Predicted: {''.join(out)}")

## GUI Layout

In [9]:
labelTitle=tk.Label(win, text="Draw Any Number", fg='black', font=fontLabel)
labelTitle.grid(row=0, column=0, columnspan=4, pady=10)

# --- Model selection centered ---
selected_model = tk.StringVar(value="CNN")
select_frame = tk.Frame(win)
select_frame.grid(row=1, column=0, columnspan=4, pady=(5, 10))

tk.Label(
    select_frame, text="Select Model:",
    fg="black", font=("fixedsys", 14, "bold")
).pack(side="left", padx=(10, 10))

dropdown = tk.OptionMenu(win, selected_model, "CNN", "DNN", "Ensemble")
dropdown.config(font=("fixedsys", 14), bg="lightyellow", width=12)
dropdown.pack(in_=select_frame, side="left")

# --- Canva --- # 
canvas=tk.Canvas(win, width=w, height=h, bg='white')
canvas.grid(row=2, column=0, columnspan=4, padx=30, pady=(15, 25))



# --- Buttons under canvas (centered) --- #
button_frame = tk.Frame(win)
button_frame.grid(row=3, column=0, columnspan=4, pady=(5, 15))

buttonSave = tk.Button(button_frame, text='Save', bg='light blue', fg='black', font=fontButton, command=save)
buttonPredict = tk.Button(button_frame, text='Predict', bg='lavender', fg='black', font=fontButton, command=predict)
buttonClear = tk.Button(button_frame, text='Clear', bg='peach puff', fg='black', font=fontButton, command=clear)
buttonExit = tk.Button(button_frame, text='Exit', bg='navajo white', fg='black', font=fontButton, command=win.destroy)

# Pack buttons inside the frame (side by side)
for b in [buttonSave, buttonPredict, buttonClear, buttonExit]:
    b.pack(side="left", padx=40)



labelStatus = tk.Label(
    win, text=f"Predicted: None  |  Mode: {selected_model.get()}",
    fg='black', font=fontLabel
)
labelStatus.grid(row=4, column=0, columnspan=4, pady=10)

# Function
canvas.bind('<B1-Motion>', eventFunction)

img=Image.new('RGB',(w,h),(0,0,0))
ImgDraw=ImageDraw.Draw(img)
 
win.mainloop()

# APPROVED✅